# FOPDT-modellering – Interaktiv Colab-notatbok

Last opp en datafil (`pid_test.txt`), juster parametre med sliders,
og last ned ferdig figur som PNG.

In [ ]:
!pip install ipywidgets
from google.colab import output
output.enable_custom_widget_manager()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.0 MB/s eta 0:00:00


In [7]:
# --- Installer ipywidgets hvis nødvendig ---
!pip install ipywidgets
from google.colab import output
output.enable_custom_widget_manager()

# --- Imports ---
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# --- Last opp fil ---
uploaded = files.upload()
filename = list(uploaded.keys())[0]

data = np.loadtxt(filename, skiprows=1)

tid_data = data[:,0]
niva_data = data[:,1]

print("Data lastet inn:", filename)
print("Antall datapunkter:", len(tid_data))

# --- Automatisk estimering ---
y0 = niva_data[0]
A_est = niva_data[-1] - y0

thresh = y0 + 0.05*A_est
L_est = tid_data[np.where(niva_data > thresh)[0][0]]

thresh63 = y0 + 0.63*A_est
T_est = tid_data[np.where(niva_data > thresh63)[0][0]] - L_est

print(f"Autoestimat: A={A_est:.2f}, T={T_est:.2f}, L={L_est:.2f}, y0={y0:.2f}")

# --- FOPDT funksjon ---
def fopdt(t, A, T, L, y0):
    return np.where(t < L, y0, y0 + A*(1 - np.exp(-(t-L)/T)))

# --- Funksjon for plot + PID formel ---
def vis_fopdt_og_pid(A, T, L, y0):
    clear_output(wait=True)
    t = np.linspace(0, tid_data.max(), 500)
    y = fopdt(t, A, T, L, y0)

    # Plot
    plt.figure(figsize=(12,6))
    plt.plot(tid_data, niva_data, 'b.', markersize=3, label="Måledata")
    plt.plot(t, y, 'r-', linewidth=2, label=f"FOPDT: A={A:.2f}, T={T:.2f}, L={L:.2f}")
    plt.xlabel("Tid")
    plt.ylabel("Nivå")
    plt.grid(True)
    plt.legend()
    plt.show()

    # PID-formler (informasjon, ikke beregning)
    display(Markdown(f"""
**Nå kan du beregne PIDF-parametrene selv!**

- Prosessforsterkning: `K = A / Δu`
- Lambda-metoden:
  - `Kp = T / (K * (λ + L))`
  - `Ti = T`
  - `Td = (L * T) / (L + λ)`

Her er de aktuelle parametrene etter sliders:
- `A = {A:.2f}`
- `T = {T:.2f}`
- `L = {L:.2f}`
- `y0 = {y0:.2f}`
"""))

# --- Sliders ---
A_slider = widgets.FloatSlider(value=A_est, min=0, max=A_est*3, step=0.5, description='A')
T_slider = widgets.FloatSlider(value=T_est, min=1, max=T_est*4, step=0.5, description='T')
L_slider = widgets.FloatSlider(value=L_est, min=0, max=L_est*3, step=0.1, description='L')
y0_slider = widgets.FloatSlider(value=y0, min=y0-10, max=y0+10, step=0.1, description='y0')

ui = widgets.VBox([A_slider, T_slider, L_slider, y0_slider])
out = widgets.interactive_output(vis_fopdt_og_pid, {'A': A_slider, 'T': T_slider, 'L': L_slider, 'y0': y0_slider})
display(ui, out)

# --- PNG eksport ---
def save_png(A, T, L, y0):
    t = np.linspace(0, tid_data.max(), 500)
    y = fopdt(t, A, T, L, y0)

    plt.figure(figsize=(12,6))
    plt.plot(tid_data, niva_data, 'b.', markersize=3)
    plt.plot(t, y, 'r-', linewidth=2)
    plt.xlabel("Tid")
    plt.ylabel("Nivå")
    plt.grid(True)
    plt.legend([f"Måledata", f"FOPDT: A={A:.2f}, T={T:.2f}, L={L:.2f}"])

    filename = "fopdt_plot.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    files.download(filename)

save_button = widgets.Button(description="📥 Last ned PNG")
def on_save_clicked(b):
    save_png(A_slider.value, T_slider.value, L_slider.value, y0_slider.value)
save_button.on_click(on_save_clicked)
display(save_button)

Saving pid_test.txt to pid_test (5).txt
Data lastet inn: pid_test (5).txt
Antall datapunkter: 316
Autoestimat: A=77.20, T=86.00, L=11.00, y0=5.30


Output()

Button(description='📥 Last ned PNG', style=ButtonStyle())

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
L

NameError: name 'L' is not defined